### Import libraries

In [1]:
from model import climate_model, losses, dot_prod_attention
from data import data_generation, combine_climate_data, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks
from inference import infer
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras
np.random.seed(1)

Using TensorFlow backend.


In [2]:
save_dir = '/Users/omernivron/Downloads/GPT_climate'

In [3]:
temp, t, token = combine_climate_data.climate_data_to_model_input('./data/t2m_monthly_averaged_ensemble_members_1989_2019.csv')

In [4]:
## create climate train/test split

In [5]:
time_tr = t[:8000]; temp_tr = temp[:8000]; token_tr = token[:8000]
time_te = t[8000:]; temp_te = temp[8000:]; token_te = token[8000:]

In [11]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [12]:
@tf.function
def train_step(decoder, optimizer_c, token_pos, time_pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_log_sig = decoder(token_pos, time_pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         print(pred_log_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_log_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_log_sig, mask

In [13]:
@tf.function
def test_step(decoder, token_pos_te, time_pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred_te, pred_log_sig_te = decoder(token_pos_te, time_pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred_te, pred_log_sig_te)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred_te, pred_log_sig_te, t_mask

In [14]:
tf.keras.backend.set_floatx('float64')

In [15]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam(0.0003)
    decoder = climate_model.Decoder(16)
    EPOCHS = 500
    batch_s  = 32
    run = 0; step = 0
    num_batches = int(temp_tr.shape[0] / batch_s)
    tf.random.set_seed(1)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer_c, net = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_climate/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)
    #https://www.tensorflow.org/guide/checkpoint
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                m_tr.reset_states(); train_loss.reset_states()
                m_te.reset_states(); test_loss.reset_states()
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, _ = batch_creator.create_batch_foxes(token_tr, time_tr, temp_tr, batch_s = 32)
                # batch_tar_tr shape := 128 X 59 = (batch_size, max_seq_len)
                # batch_pos_tr shape := 128 X 59 = (batch_size, max_seq_len)
                batch_pos_mask = masks.position_mask(batch_tok_pos_tr)
                tar_inp, tar_real, pred, pred_log_sig, mask = train_step(decoder, optimizer_c, batch_tok_pos_tr, batch_tim_pos_tr, batch_tar_tr, batch_pos_mask)

                if batch_n % 100 == 0:
                    batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, _ = batch_creator.create_batch_foxes(token_te, time_te, temp_te, batch_s = 32)
                    batch_pos_mask_te = masks.position_mask(batch_tok_pos_te)
                    tar_real_te, pred_te, pred_log_sig_te, t_mask = test_step(decoder, batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, batch_pos_mask_te)
                    helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_tr.result())
                    helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
                    manager.save()
                step += 1
                ckpt.step.assign_add(1)

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Initializing from scratch.
pos_attn1 : Tensor("decoder_1/layer_normalization_4/batchnorm/add_1:0", shape=(32, 39, 16, 16), dtype=float64)
nl_qk:  Tensor("decoder_1/nl_qk:0", shape=(32, 39, 39), dtype=float64)
tar_attn1 : Tensor("decoder_1/layer_normalization_5/batchnorm/add_1:0", shape=(32, 39, 16), dtype=float64)
tar_attn1 : Tensor("decoder_1/strided_slice_10:0", shape=(32, 39, 16, 1), dtype=float64)
tar1 : Tensor("decoder_1/B/BiasAdd:0", shape=(32, 39, 16, 16), dtype=float64)
L: Tensor("decoder_1/layer_normalization_6/batchnorm/add_1:0", shape=(32, 39, 16, 16), dtype=float64)
pos_attn1 : Tensor("decoder_1/layer_normalization_4/batchnorm/add_1:0", shape=(32, 39, 16, 16), dtype=float64)
nl_qk:  Tensor("decoder_1/nl_qk:0", shape=(32, 39, 39), dtype=float64)
tar_attn1 : Tensor("decoder_1/layer_normalization_5/batchnorm/add_1:0", shape=(32, 39, 16), dtype=float64)
tar_attn1 : Tensor("decoder_1/strided_slice_10:0", shape=(32, 39, 16, 1), dtype=float64)
tar1 : Tensor("decoder

KeyboardInterrupt: 